In [1]:
import numpy as np
import pandas as pd
import glob
import imageio
from scipy import misc
from keras.models import Model
from keras.layers import *
from keras import backend as K
from keras.optimizers import Adam
import csv
from tqdm import tqdm
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import os
%matplotlib inline

Using TensorFlow backend.


In [2]:
#图片集存放路径 
img_path = '../MusicInputFig_Eng/*' 
imgs = glob.glob(img_path) #读取该路径下所有图片 返回路径列表list

In [3]:
def imread(f):
    x = misc.imread(f, mode='RGB')
    return x.astype(np.float32) / 255 * 2 - 1

# 测试模块 载入图片

In [4]:
#得到训练集  注意: 这步很耗内存
img_train = []
index = 0
for imgPath in tqdm(iter(imgs[:1500])):
    fname = os.path.splitext(imgPath.split('/')[-1])[0] #获得歌曲名
    index = index+1
    img = imread(imgPath)
    img_train.append(img)
    del img

0it [00:00, ?it/s]/home/bob/.conda/envs/pythonDL/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
1500it [00:38, 39.34it/s]


# 正式训练 载入图片

#得到训练集  注意: 这步很耗内存
img_train = []
index = 0
with open('../Data/nameIndex.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for imgPath in tqdm(iter(imgs[:1500])):
        fname = os.path.splitext(imgPath.split('/')[-1])[0] #获得歌曲名
        row = [index, fname]
        writer.writerow(row)
        index = index+1
        img = imread(imgPath)
        img_train.append(img)
        del img

In [5]:
MusicName = 'The Glitch Mob,Mako,The Word Alive - RISE.mp3'

MusicDir = '../testMusic/music/'
FigDir = '../testMusic/musicFig/'
absolutelyMusicDir = '/home/bob/Desktop/Music_Recommend/testMusic/music/'
musicDir = '/home/bob/Music/music_dataset/'

musicPath = MusicDir + MusicName
musicFigPath = FigDir + MusicName
FigPath = musicFigPath + '.png'
#create spectrum
#figPlot.pltSave(musicPath, musicFigPath)
#read spectrum as array
img = imread(FigPath)

/home/bob/.conda/envs/pythonDL/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  


In [6]:
img_train.append(img)

In [7]:
image_train = np.array(img_train)  
x_train = image_train
del img_train

In [15]:
x_train.shape

(1501, 256, 2560, 3)

In [18]:
x_train[-1].shape

(256, 2560, 3)

In [ ]:
#-----------------------

In [9]:
img_height = x_train.shape[1]
img_weight = x_train.shape[2]

In [10]:
print(img_height, img_weight)

256 2560


In [11]:
z_dim = 128 # 隐变量维度
alpha = 0.5 # 全局互信息的loss比重
beta = 1.5 # 局部互信息的loss比重
gamma = 0.01 # 先验分布的loss比重

In [12]:
# 编码器（卷积与最大池化）
x_in = Input(shape=(img_height, img_weight, 3))
x = x_in

for i in range(4):
    x = Conv2D(int(z_dim / 2**(3-i)), 
               kernel_size=(3,3), padding='SAME')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)
    x = MaxPooling2D((2, 2))(x)

feature_map = x # 截断到这里，认为到这里是feature_map（局部特征）
feature_map_encoder = Model(x_in, x)


for i in range(2):
    x = Conv2D(z_dim,
               kernel_size=(3,3),
               padding='SAME')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

x = GlobalMaxPooling2D()(x) # 全局特征

z_mean = Dense(z_dim)(x) # 均值，也就是最终输出的编码
z_log_var = Dense(z_dim)(x) # 方差，这里都是模仿VAE的


encoder = Model(x_in, z_mean) # 总的编码器就是输出z_mean

In [13]:
# 输出编码器的特征
#100
zs = encoder.predict(x_train, verbose=True)
print(zs.mean()) # 查看均值（简单观察先验分布有没有达到效果）
print(zs.std()) # 查看方差（简单观察先验分布有没有达到效果）

1501/1501 [==============================] - 53s 36ms/step
0.0027704667
0.07766766


In [14]:
np.savetxt('/home/bob/Desktop/MusicVec.txt', zs, delimiter=',')

In [20]:
# 输出编码器的特征
#128
z1s = encoder.predict(np.expand_dims(x_train[-1], axis=0), verbose=True)
print(zs.mean()) # 查看均值（简单观察先验分布有没有达到效果）
print(zs.std()) # 查看方差（简单观察先验分布有没有达到效果）

1/1 [==============================] - 0s 157ms/step
0.0027704667
0.07766766


In [21]:
z2s = encoder.predict(np.expand_dims(img, axis=0), verbose=True)

1/1 [==============================] - 0s 22ms/step


In [71]:
# 输出编码器的特征
#135
zs = encoder.predict(x_train, verbose=True)
print(zs.mean()) # 查看均值（简单观察先验分布有没有达到效果）
print(zs.std()) # 查看方差（简单观察先验分布有没有达到效果）

1500/1500 [==============================] - 24s 16ms/step
-0.014910269
0.091249675


In [20]:
# 重参数技巧
def sampling(args):
    z_mean, z_log_var = args
    u = K.random_normal(shape=K.shape(z_mean))
    return z_mean + K.exp(z_log_var / 2) * u

# 重参数层，相当于给输入加入噪声
z_samples = Lambda(sampling)([z_mean, z_log_var])
prior_kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var))

# shuffle层，打乱第一个轴
def shuffling(x):
    idxs = K.arange(0, K.shape(x)[0])
    idxs = K.tf.random_shuffle(idxs)
    return K.gather(x, idxs)

In [21]:
# 与随机采样的特征拼接（全局）
z_shuffle = Lambda(shuffling)(z_samples)
z_z_1 = Concatenate()([z_samples, z_samples])
z_z_2 = Concatenate()([z_samples, z_shuffle])

# 与随机采样的特征拼接（局部）
feature_map_shuffle = Lambda(shuffling)(feature_map)
z_samples_repeat = RepeatVector(16 * 160)(z_samples)
z_samples_map = Reshape((16, 160, z_dim))(z_samples_repeat)
z_f_1 = Concatenate()([z_samples_map, feature_map])
z_f_2 = Concatenate()([z_samples_map, feature_map_shuffle])

# 全局判别器
z_in = Input(shape=(z_dim*2,))
z = z_in
z = Dense(z_dim, activation='relu')(z)
z = Dense(z_dim, activation='relu')(z)
z = Dense(z_dim, activation='relu')(z)
z = Dense(1, activation='sigmoid')(z)

GlobalDiscriminator = Model(z_in, z)

z_z_1_scores = GlobalDiscriminator(z_z_1)
z_z_2_scores = GlobalDiscriminator(z_z_2)
global_info_loss = - K.mean(K.log(z_z_1_scores + 1e-6) + K.log(1 - z_z_2_scores + 1e-6))

# 局部判别器
z_in = Input(shape=(None, None, z_dim*2))
z = z_in
z = Dense(z_dim, activation='relu')(z)
z = Dense(z_dim, activation='relu')(z)
z = Dense(z_dim, activation='relu')(z)
z = Dense(1, activation='sigmoid')(z)

LocalDiscriminator = Model(z_in, z)

z_f_1_scores = LocalDiscriminator(z_f_1)
z_f_2_scores = LocalDiscriminator(z_f_2)
local_info_loss = - K.mean(K.log(z_f_1_scores + 1e-6) + K.log(1 - z_f_2_scores + 1e-6))

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 16, 160, 512), (None, 1, 10, 512)]

# 训练过程  如果没有训练好的模型文件 则运行以下代码 若有模型文件 跳过下面两个代码块

# 用来训练的模型
model_train = Model(x_in, [z_z_1_scores, z_z_2_scores, z_f_1_scores, z_f_2_scores])
model_train.add_loss(alpha * global_info_loss + beta * local_info_loss + gamma * prior_kl_loss)
model_train.compile(optimizer=Adam(1e-3))

model_train.fit(x_train, epochs=100, batch_size=100)
#model_train.save_weights('image.weights')

# 载入训练好的模型文件

In [16]:
# 用来训练的模型
model_train = Model(x_in, [z_z_1_scores, z_z_2_scores, z_f_1_scores, z_f_2_scores])
model_train.add_loss(alpha * global_info_loss + beta * local_info_loss + gamma * prior_kl_loss)
model_train.compile(optimizer=Adam(1e-3))

### Train model

In [10]:
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 2560, 3)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 2560, 32)     896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 2560, 32)     128       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256, 2560, 32)     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 1280, 32)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 1280, 64)     18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 1280, 64)     256       
__________

In [ ]:
from keras.utils import plot_model
plot_model(model_train, to_file='model.png')

In [ ]:
model_train.summary()

In [ ]:
#model_train.save_weights('Music_Recommend.weights')
#model_train.save('Music_Model_100.h5')
model_train = model_train.load_weights('Music_Recommend.weights', by_name=False)

In [18]:
print(type(model_train), type(history))


NameError: name 'model_train' is not defined

In [ ]:
#history = model_train
# list all data in history
print(history.history.keys())
# summarize history for loss
plt.plot(history.history['loss'])
#plt.plot(history.history['epochs'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
plt.show()

### zs 就是编码器根据输入得出的每个图片的特征向量

In [13]:
zs.shape

(1500, 128)

In [14]:
np.savetxt('../Data/MusicVec.txt', zs, delimiter=',')
#zs = np.loadtxt('../Data/MusicVec.txt', delimiter=',')

# 以下部分以迁移至Music_Recommend

In [ ]:
#参宿为输出图片路径 这里是指当前路径下pic文件夹下test.jpg
#sample_knn('pic/test')

In [ ]:
def chooseSimMus():
    musInputList = [236, 344, 407, 426, 744, 214, 1977, 1675, 1535, 1371]
    n = len(musInputList)
    #n = 5
    topn = 10
    #figure1 = np.zeros((img_height*n, img_weight*topn, 3))
    #figure2 = np.zeros((img_height*n, img_weight*topn, 3))
    zs_ = zs / (zs**2).sum(1, keepdims=True)**0.5
    similar_list = []
    for i , one in zip(range(n), musInputList):
        #one = 0 #测试选择0号
        #one = np.random.choice(len(x_train)) #随机选择一张图的序号
        idxs = ((zs**2).sum(1) + (zs[one]**2).sum() - 2 * np.dot(zs, zs[one])).argsort()[:topn]
        similar = [one, idxs.tolist()]
        similar_list.append(similar)
    return similar_list

In [15]:
def randomSimMus():
    #musInputList = [236, 344, 407, 426, 744, 214, 1977, 1675, 1535, 1371]
    n = 30
    topn = 11
    #figure1 = np.zeros((img_height*n, img_weight*topn, 3))
    #figure2 = np.zeros((img_height*n, img_weight*topn, 3))
    zs_ = zs / (zs**2).sum(1, keepdims=True)**0.5
    similar_list = []
    for i in range(n):
        one = np.random.choice(len(x_train)) #随机选择一张图的序号
        idxs = ((zs**2).sum(1) + (zs[one]**2).sum() - 2 * np.dot(zs, zs[one])).argsort()[:topn]
        similar = [one, idxs.tolist()]
        similar_list.append(similar)
    return similar_list

In [16]:
#similar_list = chooseSimMus()
similar_list = randomSimMus()

lena = mpimg.imread('../Data/music_l2.png')
plt.figure(figsize=(30, 15))
plt.imshow(lena) # 显示图片
plt.axis('off') # 不显示坐标轴
plt.show()

In [17]:
similar_list

[[880, [880, 1442, 457, 771, 1162, 656, 254, 80, 320, 1014, 1337]],
 [546, [546, 110, 975, 1128, 164, 268, 1047, 705, 1191, 1105, 187]],
 [585, [585, 1138, 545, 471, 1148, 822, 1433, 568, 403, 623, 152]],
 [605, [605, 678, 885, 711, 1139, 1187, 326, 491, 733, 98, 753]],
 [544, [544, 300, 1079, 109, 767, 1236, 1214, 115, 436, 961, 219]],
 [1063, [1063, 836, 1250, 1470, 1265, 1366, 381, 1093, 385, 1236, 387]],
 [490, [490, 965, 225, 659, 939, 896, 543, 33, 587, 1025, 180]],
 [895, [895, 143, 192, 908, 122, 736, 510, 1176, 807, 1104, 526]],
 [835, [835, 91, 1456, 893, 851, 331, 899, 1057, 653, 18, 607]],
 [1260, [1260, 205, 268, 40, 581, 1128, 980, 1080, 258, 1227, 144]],
 [1344, [1344, 1411, 1025, 1388, 570, 535, 502, 1378, 1180, 534, 870]],
 [1463, [1463, 643, 27, 1112, 970, 1152, 163, 1333, 1139, 414, 768]],
 [975, [975, 1191, 1105, 546, 1047, 705, 94, 110, 220, 1435, 267]],
 [935, [935, 689, 296, 1421, 859, 262, 873, 410, 105, 1343, 951]],
 [778, [778, 1438, 991, 907, 461, 502, 910, 7

In [18]:
similar_dic = {}
for img_id, name_list in similar_list:
    similar_dic[img_id] = name_list
similar_dic

{880: [880, 1442, 457, 771, 1162, 656, 254, 80, 320, 1014, 1337],
 546: [546, 110, 975, 1128, 164, 268, 1047, 705, 1191, 1105, 187],
 585: [585, 1138, 545, 471, 1148, 822, 1433, 568, 403, 623, 152],
 605: [605, 678, 885, 711, 1139, 1187, 326, 491, 733, 98, 753],
 544: [544, 300, 1079, 109, 767, 1236, 1214, 115, 436, 961, 219],
 1063: [1063, 836, 1250, 1470, 1265, 1366, 381, 1093, 385, 1236, 387],
 490: [490, 965, 225, 659, 939, 896, 543, 33, 587, 1025, 180],
 895: [895, 143, 192, 908, 122, 736, 510, 1176, 807, 1104, 526],
 835: [835, 91, 1456, 893, 851, 331, 899, 1057, 653, 18, 607],
 1260: [1260, 205, 268, 40, 581, 1128, 980, 1080, 258, 1227, 144],
 1344: [1344, 1411, 1025, 1388, 570, 535, 502, 1378, 1180, 534, 870],
 1463: [1463, 643, 27, 1112, 970, 1152, 163, 1333, 1139, 414, 768],
 975: [975, 1191, 1105, 546, 1047, 705, 94, 110, 220, 1435, 267],
 935: [935, 689, 296, 1421, 859, 262, 873, 410, 105, 1343, 951],
 778: [778, 1438, 991, 907, 461, 502, 910, 707, 1000, 1180, 98],
 747: [7

In [ ]:
similar_arr = pd.DataFrame(similar_dic)

In [ ]:
similar_arr.to_csv('../Data/SimilaryMusicIndex.csv', index=False)
similar_arr

In [ ]:
img_ind_data = pd.read_csv('../Data/SimilaryMusicIndex.csv')

In [ ]:
tarImgs = img_ind_data.columns.tolist()

In [ ]:
img_data = pd.read_csv('../Data/nameIndex.csv', names=['id', 'name'])
img_data.head()

In [ ]:
for img_id in tarImgs:
    targetImg = img_data[img_data['id']==int(img_id)]
    targetImgName = targetImg['name'].values[0]
    simImgs = img_ind_data[img_id].values.tolist()
    print("Input Image name is:"+'\n', targetImgName)
    print('Similar Image name is:')
    for sims in simImgs:
        #print(sims)
        simImage = img_data[img_data['id']==int(sims)]
        simName = simImage['name'].values[0]
        print('\t' + os.path.join('/home/bob/Music/music_dataset', simName))
    print('\n')